<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-1/blob/main/ex09_Techniques_avanc%C3%A9es_de_requ%C3%AAte_avec_utilisation_de_CASE_et_de_sous_requ%C3%AAte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ex09-Techniques avancées de requête avec utilisation de CASE et de sous-requête

L'expression SQLite CASE évalue une liste de conditions et renvoie une expression basée sur le résultat de l'évaluation. 

L'expression CASE est similaire à l'instruction IF-THEN-ELSE dans d'autres langages de programmation. 

Vous pouvez utiliser l'instruction CASE dans n'importe quelle clause ou instruction qui accepte une expression valide. 

Par exemple, vous pouvez utiliser l'instruction CASE dans des clauses telles que WHERE, ORDER BY, HAVING, IN, SELECT et des instructions telles que SELECT, UPDATE et DELETE. 

Pour en savoir plus, consultez http://www.sqlitetutorial.net/sqlite-case/.

Une sous-requête, en termes simples, est une requête écrite dans le cadre d'une instruction plus grande. 

Considérez-la comme une instruction SELECT dans une autre. 
Le résultat du SELECT interne peut ensuite être utilisé dans la requête externe.

Dans ce notebook, nous combinons ces deux techniques d'interrogation pour calculer le ruissellement saisonnier à partir des données année-mois de la table ***rch***.

In [1]:
%load_ext sql

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [3]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

If you do not remember the tables in the demo data, you can always use the following command to query.

In [4]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Exploration de la table rch

Nous pouvons constater que la table rch contient des données de séries chronologiques avec l'année et le mois pour chaque tronçon de rivière. 

Par conséquent, il est logique de calculer des statistiques saisonnières.


In [5]:
%sql SELECT * From rch LIMIT 3

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,ORGN_INkg,ORGN_OUTkg,ORGP_INkg,ORGP_OUTkg,NO3_INkg,NO3_OUTkg,NH4_INkg,NH4_OUTkg,NO2_INkg,NO2_OUTkg,MINP_INkg,MINP_OUTkg,CHLA_INkg,CHLA_OUTkg,CBOD_INkg,CBOD_OUTkg,DISOX_INkg,DISOX_OUTkg,SOLPST_INmg,SOLPST_OUTmg,SORPST_INmg,SORPST_OUTmg,REACTPSTmg,VOLPSTmg,SETTLPSTmg,RESUSP_PSTmg,DIFFUSEPSTmg,REACBEDPSTmg,BURYPSTmg,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
1,1981,1,146.34376525878906,146.2524871826172,0.09128088504076004,0.0,2.3320461650655488e-07,61619.46484375,155.3719024658203,0.016086287796497345,0.0,0.04825887829065323,0.0,362.0486755371094,361.8135070800781,203.620849609375,421.18377685546875,0.0,23.0184326171875,0.016107244417071342,0.0,1.1839052307949238e-11,0.0,0.0,0.0,5627225.0,5623486.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,806.0157470703125,0.0,0.0,0.0
2,1981,1,96.22569274902344,96.18285369873047,0.042821284383535385,0.0,1.6426764659627224e-07,0.0,0.0,0.01363153662532568,0.0,0.04089462012052536,0.0,315.60052490234375,315.4579772949219,0.0,127.00502014160156,0.0,0.0,0.013656049966812134,0.0,4.136972177163251e-16,0.0,0.0,0.0,3757606.5,3698301.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,442.4630126953125,0.0,0.0,0.0
3,1981,1,11.952718734741211,11.861368179321289,0.09135182946920395,0.0,2.0325823868461157e-07,2.0325823868461157e-07,6.595060941805286e-09,0.01146628800779581,0.0,0.03439886122941971,0.009118038229644299,48.2963752746582,47.93150329589844,0.0,62.467620849609375,0.0,0.0,0.011485105380415916,0.0,5.941028783295818e-14,0.0,0.0,0.0,360979.90625,456115.90625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.39912414550781,0.009118038229644299,0.0,0.0


### 3. Calcul du ruissellement saisonnier

Il y a deux étapes clés:
> (1) utiliser le CASE et la sous-requête pour convertir des mois en saisons dénommées; <br>
> (2) calculer la moyenne saisonnière avec des fonctions agrégées sur les groupes.

De plus, nous utilisons également un autre mot-clé de filtre ***BETWEEN*** pour regrouper et présenter les mois sous forme de saisons.

In [6]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM(
SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch)
GROUP BY RCH, Quarter

Done.


RCH,Quarter,Runoff
1,DJF,99.20499053531223
1,JJA,1405.262297990587
1,MAM,559.7469320191277
1,SON,454.73798531426326
2,DJF,56.328539085388186
2,JJA,773.6642798105876
2,MAM,203.2428277624978
2,SON,297.9343263414171
3,DJF,32.838328109847176
3,JJA,166.31038238737318


### Conclusion

Parfois, nous pouvons avoir besoin de construire des requêtes complexes qui vont au-delà d'une jointure de table ou d'une requête SELECT de base. 

Par exemple, nous pouvons avoir besoin d'écrire une requête qui utilise les résultats d'autres requêtes comme entrées (par exemple, SUBQUERY). 
Ou nous pouvons avoir besoin de regrouper des valeurs numériques en catégories avant de les compter (c'est-à-dire CASE).

Dans ce Notebook, nous avons exploré une collection de fonctions et d'options SQL essentielles pour résoudre des problèmes plus complexes. 

Nous savons désormais ajouter des sous-requêtes pour fournir un contrôle plus fin sur le filtrage ou le prétraitement des données avant de les analyser dans la requête principale.